In [ ]:
!pip -q install geopandas shapely pyproj fiona pandas


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 프로젝트 루트 (필요하면 이 한 줄만 수정)
ROOT = '/content/drive/MyDrive/colab/firecast'

# 데이터 경로 (01 노트북에서 만든 구조 기준)
FIRE_DIR = f'{ROOT}/data/raw/fires/FRT000102_42'
OUT_DIR  = f'{ROOT}/data/processed'

import os, glob
os.makedirs(OUT_DIR, exist_ok=True)

print('ROOT:', ROOT)
print('FIRE_DIR:', FIRE_DIR)
print('OUT_DIR:', OUT_DIR)


In [ ]:
from dataclasses import dataclass
from typing import List
import pandas as pd
import geopandas as gpd

@dataclass
class WeatherStation:
    station_id: int       # 104, 105 같은 번호
    name_kr: str          # 북강릉, 강릉
    name_en: str          # Bukgangneung, Gangneung
    lat: float            # 위도 (WGS84)
    lon: float            # 경도 (WGS84)

class WeatherStationRegistry:
    """
    관측소 정보를 관리하는 레지스트리.
    - 지금은 104(북강릉), 105(강릉)만 하드코딩
    - 나중에 from_csv(...) 같은 메서드 추가해서 확장 가능
    """
    def __init__(self, stations: List[WeatherStation]):
        self.stations = stations

    @classmethod
    def default_kma_gangneung(cls):
        """현재 알고 있는 북강릉/강릉 관측소를 기본 레지스트리로 생성"""
        stations = [
            WeatherStation(
                station_id=104,
                name_kr='북강릉',
                name_en='Bukgangneung',
                lat=37.80456,
                lon=128.85535,
            ),
            WeatherStation(
                station_id=105,
                name_kr='강릉',
                name_en='Gangneung',
                lat=37.75146,
                lon=128.89098,
            ),
        ]
        return cls(stations)

    def to_geodataframe(self, crs: str = 'EPSG:4326') -> gpd.GeoDataFrame:
        """관측소 리스트를 GeoDataFrame으로 변환 (기본: WGS84)"""
        df = pd.DataFrame([s.__dict__ for s in self.stations])
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df['lon'], df['lat']),
            crs='EPSG:4326'
        )
        if crs != 'EPSG:4326':
            gdf = gdf.to_crs(crs)
        return gdf


def attach_nearest_station(
    fire_gdf: gpd.GeoDataFrame,
    registry: WeatherStationRegistry,
    distance_col: str = 'dist_m'
) -> gpd.GeoDataFrame:
    """
    산불 지점(fire_gdf)에 관측소 레지스트리(registry)를 이용해
    유클리드 거리 기준 가장 가까운 관측소를 붙이는 함수.

    - 거리 단위: m (EPSG:5179 기준)
    - 반환값: fire_gdf + [station_id, name_kr, name_en, dist_m]
    """
    if fire_gdf.crs is None:
        raise ValueError('fire_gdf.crs 가 None입니다. CRS를 먼저 지정해 주세요.')

    # 1) 관측소 GeoDataFrame (WGS84 기준 생성)
    stations_wgs84 = registry.to_geodataframe(crs='EPSG:4326')

    # 2) 산불 좌표를 WGS84로 맞추기 (필요 시)
    if fire_gdf.crs.to_string() != 'EPSG:4326':
        fires_wgs84 = fire_gdf.to_crs('EPSG:4326')
    else:
        fires_wgs84 = fire_gdf

    # 3) 거리 계산을 위해 미터 기반 투영 좌표계로 변환 (예: EPSG:5179)
    fires_proj    = fires_wgs84.to_crs('EPSG:5179')
    stations_proj = stations_wgs84.to_crs('EPSG:5179')

    # 4) 최근접 spatial join
    joined = gpd.sjoin_nearest(
        fires_proj,
        stations_proj[['station_id', 'name_kr', 'name_en', 'geometry']],
        how='left',
        distance_col=distance_col
    )

    # 5) 다시 원래 fire_gdf의 CRS로 돌려서 반환
    if fire_gdf.crs.to_string() != 'EPSG:5179':
        joined = joined.to_crs(fire_gdf.crs)

    return joined


In [ ]:
import geopandas as gpd

# shapefile 한 개 찾기
shp_list = glob.glob(os.path.join(FIRE_DIR, '*.shp'))
assert len(shp_list) == 1, f'*.shp 파일이 1개가 아닙니다: {shp_list}'
FIRE_SHP = shp_list[0]
print('FIRE_SHP:', FIRE_SHP)

fires = gpd.read_file(FIRE_SHP)
print('fires.shape:', fires.shape)
print('fires.crs:', fires.crs)


fires['YEAR'] = fires['OCCRR_DTM'].astype(str).str[:4]
fires = fires[fires['YEAR'].isin(['2020', '2021'])].copy()
fires = fires.drop(columns=['YEAR'])

print('filtered fires.shape:', fires.shape)

fires.head()

In [ ]:
# 1) 레지스트리 생성 (104: 북강릉, 105: 강릉)
registry = WeatherStationRegistry.default_kma_gangneung()

# 2) 최근접 관측소 붙이기
fires_with_station = attach_nearest_station(
    fire_gdf=fires,
    registry=registry,
    distance_col='dist_m'   # 단위: meter
)

print('result shape:', fires_with_station.shape)
fires_with_station[['station_id', 'name_kr', 'name_en', 'dist_m']].head()


In [ ]:
# 관측소별 건수
print('[station_id counts]')
print(fires_with_station['station_id'].value_counts())

# 거리 통계
print('\n[distance stats (m)]')
print(fires_with_station['dist_m'].describe())


In [ ]:
import os

out_parquet = os.path.join(OUT_DIR, 'fires_with_manual_station.parquet')
fires_with_station.to_parquet(out_parquet, index=False)
print('saved parquet ->', out_parquet)

# 필요하면 CSV나 Shapefile로도 저장 가능
fires_with_station.to_file(os.path.join(OUT_DIR, 'fires_with_manual_station.shp'))
print('saved shapefile ->', os.path.join(OUT_DIR, 'fires_with_manual_station.shp'))
